In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class LogisticRegression(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.Sigmoid()
            # we are using Sigmoid function as an activation function that the output will be in range [0, 1]
        )
    
    def forward(self, X):
        y_pred = self.linear(X)
        return y_pred

In [3]:
# MNIST dataset (images and labels)
train_data = datasets.MNIST(root='../../data', train=True, transform=transforms.ToTensor()) # download before using
test_data = datasets.MNIST(root='../../data', train=False, transform=transforms.ToTensor())

# Using DalaLoader to make datasets iterable
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

print("Number of pictures for train:", len(train_loader))

Number of pictures for train: 938


In [4]:
def train(model, epoches, learning_rate):
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()  
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    total_num = len(train_loader)
    for epoch in range(epoches):
        for i, (images, labels) in enumerate(train_loader):
            images = images.view(-1, 28*28)
            
            y_pred = model(images)
            loss = criterion(y_pred, labels)
            
            # Zero gradient value
            optimizer.zero_grad()
            # Algorithm error backpropagation
            loss.backward()
            # Update our weigths
            optimizer.step()

            if (i+1) % 200 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epoches, i+1, total_num, loss.item()))

In [5]:
def test(model):
    correct = 0
    total = 0
    
    # In test phase, we don't need to compute gradients (for memory efficiency) and use torch.no_grad()
    # More: https://pytorch.org/docs/stable/autograd.html#locally-disable-grad
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.reshape(-1, 28*28)
            y_pred = model(images)
            _, predicted = torch.max(y_pred.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()

        print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

In [6]:
logreg = LogisticRegression(28*28, 10)

In [7]:
train(logreg, 10, 1e-3)

Epoch [1/10], Step [200/938], Loss: 2.2918
Epoch [1/10], Step [400/938], Loss: 2.2749
Epoch [1/10], Step [600/938], Loss: 2.2670
Epoch [1/10], Step [800/938], Loss: 2.2698
Epoch [2/10], Step [200/938], Loss: 2.2334
Epoch [2/10], Step [400/938], Loss: 2.2252
Epoch [2/10], Step [600/938], Loss: 2.2173
Epoch [2/10], Step [800/938], Loss: 2.2227
Epoch [3/10], Step [200/938], Loss: 2.1919
Epoch [3/10], Step [400/938], Loss: 2.1812
Epoch [3/10], Step [600/938], Loss: 2.1717
Epoch [3/10], Step [800/938], Loss: 2.1486
Epoch [4/10], Step [200/938], Loss: 2.1552
Epoch [4/10], Step [400/938], Loss: 2.1585
Epoch [4/10], Step [600/938], Loss: 2.1160
Epoch [4/10], Step [800/938], Loss: 2.1267
Epoch [5/10], Step [200/938], Loss: 2.0946
Epoch [5/10], Step [400/938], Loss: 2.0819
Epoch [5/10], Step [600/938], Loss: 2.1127
Epoch [5/10], Step [800/938], Loss: 2.0693
Epoch [6/10], Step [200/938], Loss: 2.0782
Epoch [6/10], Step [400/938], Loss: 2.0566
Epoch [6/10], Step [600/938], Loss: 2.0729
Epoch [6/10

In [8]:
test(logreg)

Accuracy of the model on the 10000 test images: 79 %


#### So, using logistic regression approach we achieved a good result. Without any doubts, convolutional neural networks (CNNs) would be better for classifying images. 